## Molecules

Molecules will render a 3D charge field model of molecular matter 
from atoms selected by the user.  

That's the plan anyway. At first I'll try making atoms. Molecules 
is Version 2 of Atom Builder (AB), replacing AB's 19 i-loop slots 
with a 19 slot csv data file. The expected faster renderings and 
efficiencies gained should allow us to build multiple atoms in 
molecular configurations. 

I might end up using the json data file instead. I think I'll try
reusing most of AB's code, such as the periodic table. This time 
around we'll use Cr6's original Slotlayout (SL) diagram.   

Molecules is intended to introduce Charge field theory - see
the notebook's end cell's source references.
                                            

In [1]:
from pythreejs import *
from IPython.display import display
from ipywidgets import HBox, VBox, Button, ButtonStyle, GridspecLayout
import ipywidgets as widgets
from math import sqrt, tan
from random import random , randint , seed

In [2]:
# A few values used
phi = (1+sqrt(5))/2  # calculating phi-based-pi, piG.
piG = 4/sqrt(phi)    # piG is used instead of pi.
pRadius = 0.05 # proton radius, used by all atomic distances
caroSecs = 5.0 # Problem Here # time in secs for the carousel to make one rotation

In [3]:
# Sizing the view, camera and adding lights
view_width = 600
view_height = 400
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 0, 1], aspect=view_width/view_height)
#camera = PerspectiveCamera(position=[5, 0, 0], up=[1, 0, 0], aspect=view_width/view_height)
camera = PerspectiveCamera(position=[5, 0, 0], up=[0, 1, 0], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 5, 5])
ambient_light = AmbientLight()

In [21]:
import pandas as pd

import numpy as np

#pd.set_option('max_columns', 4, 'max_rows', 10, 'max_colwidth', 12)

filename = '../data/Cr6-Elements.csv'
    
df_raw = pd.read_csv(filename) 

# Uncomment df_raw or df_raw.info() to view
# df_raw
# df_raw.info()

In [22]:
df = df_raw.copy()

# Make changes to the dataFrame created by pd.read_csv(filename)'s 
# method defaults. The data type constrains how you may work with 
# the data.
 
# Make "SlotOrien" and "CanBind" boolean.
df["CanBind"] = np.where(df["CanBind"] == "Y", True, False)
df["SlotOrien"] = df["SlotOrien"].map({"H":True, "V":False}) 

# Slot coordinates need to be float
df["SlotX"] = df["SlotX"].astype("float")
df["SlotY"] = df["SlotY"].astype("float")
df["SlotZ"] = df["SlotZ"].astype("float")

# Do not include the SlotLayout columns.
df = df.iloc[:,:19]

# Uncomment df or df.info() to view
# df
# df.info()

In [23]:
"""
'elements'is a list of tuples used by the program to  
identify or re-select a desired atom from the gui 
dropdown or periodic table. Also used as the optional
3D atom's atomic label. 'elements' is taken from the 
df['AtomicTuple'] column.  

Starting with 1710 rows. After drop_duplicates, the 
'AtomicTuple' row index increments by 19's, 0 to 1709, 
19x expanded. The Series.array method renumbers the 
index back to the desired 0-89. The two steps are 
chained.

Next, atomicTups has been read as a list of strings. 
Those strings are converted into tuples by using 
the map and eval methods.

I found the conversion by searching "How to convert 
a list of strings into tuples in python?". It works. 
"""
atomicTups = df['AtomicTuple'].drop_duplicates().array

elements = list(map(eval, atomicTups))

#elements

In [24]:
# The periodic table's 90 buttons. Each button includes: Atomic type color, clist,
# worked in steps first-fifth. Atomic symbol, dlist. Layout width and layout height

# First, the ten Atomic types, tenATypes. tenATypes = list(df['Atomic ... works ok, 
#the list is simpler.
#tenATypes = []
#tenATypes = list(df['AtomicType'].drop_duplicates())  # 1710 rows down to 10 rows 
tenATypes = ['nonmetal','noble_gas','alkali_metal','alkaline_earth','metalloid',
             'halogen','basic_metal','transition_metal','lanthanide','actinide']

# Second - The 10 atomic type colors, tenColors. Lined up with tenATypes
tenColors = ['skyblue','cyan','tomato','orange','springgreen','powderblue','palegreen',
         'yellow','thistle','violet']

# Third, zip together the type/color dictionary
typeColorDict = dict(zip(tenATypes, tenColors))
#typeColorDict

# Fourth - creating an ('Atomicnumber','AtomicType') 90 item tuple list.
df['allATypes'] = list(zip(df.AtomicNumber, df.AtomicType))
ATypes = df['allATypes'].drop_duplicates().array 
#ATypes

# Fifth - Build the list of 90 colors, clist
clist = []
for i in range(90):
    clist.append(typeColorDict[ATypes[i][1]])
#clist

# The atomic symbols for the Periodic table. 
dlist = df['AtomicSymbol'].drop_duplicates().array

# Defining the buttons by adding the atomic symbols and colors to the buttons
a1 = Button(description=dlist[0], style=dict(button_color=clist[0]))
a2 = Button(description=dlist[1], style=dict(button_color=clist[1]))
a3 = Button(description=dlist[2], style=dict(button_color=clist[2]))
a4 = Button(description=dlist[3], style=dict(button_color=clist[3]))
a5 = Button(description=dlist[4], style=dict(button_color=clist[4]))
a6 = Button(description=dlist[5], style=dict(button_color=clist[5]))
a7 = Button(description=dlist[6], style=dict(button_color=clist[6]))
a8 = Button(description=dlist[7], style=dict(button_color=clist[7]))
a9 = Button(description=dlist[8], style=dict(button_color=clist[8]))
a10 = Button(description=dlist[9], style=dict(button_color=clist[9]))
a11 = Button(description=dlist[10], style=dict(button_color=clist[10]))
a12 = Button(description=dlist[11], style=dict(button_color=clist[11]))
a13 = Button(description=dlist[12], style=dict(button_color=clist[12]))
a14 = Button(description=dlist[13], style=dict(button_color=clist[13]))
a15 = Button(description=dlist[14], style=dict(button_color=clist[15]))
a16 = Button(description=dlist[15], style=dict(button_color=clist[15]))
a17 = Button(description=dlist[16], style=dict(button_color=clist[16]))
a18 = Button(description=dlist[17], style=dict(button_color=clist[17]))
a19 = Button(description=dlist[18], style=dict(button_color=clist[18]))
a20 = Button(description=dlist[19], style=dict(button_color=clist[19]))
a21 = Button(description=dlist[20], style=dict(button_color=clist[20]))
a22 = Button(description=dlist[21], style=dict(button_color=clist[22]))
a23 = Button(description=dlist[22], style=dict(button_color=clist[22]))
a24 = Button(description=dlist[23], style=dict(button_color=clist[23]))
a25 = Button(description=dlist[24], style=dict(button_color=clist[24]))
a26 = Button(description=dlist[25], style=dict(button_color=clist[25]))
a27 = Button(description=dlist[26], style=dict(button_color=clist[26]))
a28 = Button(description=dlist[27], style=dict(button_color=clist[27]))
a29 = Button(description=dlist[28], style=dict(button_color=clist[28]))
a30 = Button(description=dlist[29], style=dict(button_color=clist[29]))
a31 = Button(description=dlist[30], style=dict(button_color=clist[30]))
a32 = Button(description=dlist[31], style=dict(button_color=clist[31]))
a33 = Button(description=dlist[32], style=dict(button_color=clist[32]))
a34 = Button(description=dlist[33], style=dict(button_color=clist[33]))
a35 = Button(description=dlist[34], style=dict(button_color=clist[34]))
a36 = Button(description=dlist[35], style=dict(button_color=clist[35]))
a37 = Button(description=dlist[36], style=dict(button_color=clist[36]))
a38 = Button(description=dlist[37], style=dict(button_color=clist[37]))
a39 = Button(description=dlist[38], style=dict(button_color=clist[38]))
a40 = Button(description=dlist[39], style=dict(button_color=clist[39]))
a41 = Button(description=dlist[40], style=dict(button_color=clist[40]))
a42 = Button(description=dlist[41], style=dict(button_color=clist[41]))
a43 = Button(description=dlist[42], style=dict(button_color=clist[42]))
a44 = Button(description=dlist[43], style=dict(button_color=clist[43]))
a45 = Button(description=dlist[44], style=dict(button_color=clist[44]))
a46 = Button(description=dlist[45], style=dict(button_color=clist[45]))
a47 = Button(description=dlist[46], style=dict(button_color=clist[46]))
a48 = Button(description=dlist[47], style=dict(button_color=clist[47]))
a49 = Button(description=dlist[48], style=dict(button_color=clist[48]))
a50 = Button(description=dlist[49], style=dict(button_color=clist[49]))
a51 = Button(description=dlist[50], style=dict(button_color=clist[50]))
a52 = Button(description=dlist[51], style=dict(button_color=clist[51]))
a53 = Button(description=dlist[52], style=dict(button_color=clist[52]))
a54 = Button(description=dlist[53], style=dict(button_color=clist[53]))
a55 = Button(description=dlist[54], style=dict(button_color=clist[54]))
a56 = Button(description=dlist[55], style=dict(button_color=clist[55]))
a57 = Button(description=dlist[56], style=dict(button_color=clist[56]))
a58 = Button(description=dlist[57], style=dict(button_color=clist[57]))
a59 = Button(description=dlist[58], style=dict(button_color=clist[58]))
a60 = Button(description=dlist[59], style=dict(button_color=clist[59]))
a61 = Button(description=dlist[60], style=dict(button_color=clist[60]))
a62 = Button(description=dlist[61], style=dict(button_color=clist[61]))
a63 = Button(description=dlist[62], style=dict(button_color=clist[62]))
a64 = Button(description=dlist[63], style=dict(button_color=clist[63]))
a65 = Button(description=dlist[64], style=dict(button_color=clist[64]))
a66 = Button(description=dlist[65], style=dict(button_color=clist[65]))
a67 = Button(description=dlist[66], style=dict(button_color=clist[66]))
a68 = Button(description=dlist[67], style=dict(button_color=clist[67]))
a69 = Button(description=dlist[68], style=dict(button_color=clist[68]))
a70 = Button(description=dlist[69], style=dict(button_color=clist[69]))
a71 = Button(description=dlist[70], style=dict(button_color=clist[70]))
a72 = Button(description=dlist[71], style=dict(button_color=clist[71]))
a73 = Button(description=dlist[72], style=dict(button_color=clist[72]))
a74 = Button(description=dlist[73], style=dict(button_color=clist[73]))
a75 = Button(description=dlist[74], style=dict(button_color=clist[74]))
a76 = Button(description=dlist[75], style=dict(button_color=clist[75]))
a77 = Button(description=dlist[76], style=dict(button_color=clist[76]))
a78 = Button(description=dlist[77], style=dict(button_color=clist[77]))
a79 = Button(description=dlist[78], style=dict(button_color=clist[78]))
a80 = Button(description=dlist[79], style=dict(button_color=clist[79]))
a81 = Button(description=dlist[80], style=dict(button_color=clist[80]))
a82 = Button(description=dlist[81], style=dict(button_color=clist[81]))
a83 = Button(description=dlist[82], style=dict(button_color=clist[82]))
a84 = Button(description=dlist[83], style=dict(button_color=clist[83]))
a85 = Button(description=dlist[84], style=dict(button_color=clist[84]))
a86 = Button(description=dlist[85], style=dict(button_color=clist[85]))
a87 = Button(description=dlist[86], style=dict(button_color=clist[86]))
a88 = Button(description=dlist[87], style=dict(button_color=clist[87]))
a89 = Button(description=dlist[88], style=dict(button_color=clist[88]))
a90 = Button(description=dlist[89], style=dict(button_color=clist[89]))

# Creating the list of buttons
abttns = [a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,
               a20,a21,a22,a23,a24,a25,a26,a27,a28,a29,a30,a31,a32,a33,a34,a35,a36,a37,a38,a39,
               a40,a41,a42,a43,a44,a45,a46,a47,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57,a58,a59,
               a60,a61,a62,a63,a64,a65,a66,a67,a68,a69,a70,a71,a72,a73,a74,a75,a76,a77,a78,a79,
               a80,a81,a82,a83,a84,a85,a86,a87,a88,a89,a90]
# Use the button list in a loop to add each button's border, layout width and height.
i=0
while i < 90 :
    abttns[i].layout = widgets.Layout(border = '1px solid black') 
    abttns[i].layout.width = '100%'
    abttns[i].layout.height = '100%'  #'40px'
    i += 1
    
# Layout the 90 buttons in periodic table form
grid = GridspecLayout(10, 18)

grid[0,0] = abttns[0]
grid[0,17] = abttns[1]
grid[1,0] = abttns[2]
grid[1,1] = abttns[3]
p=12
while p < 18:
    grid[1,p] = abttns[p-8]
    p += 1
grid[2,0] = abttns[10]
grid[2,1] = abttns[11]
n=12
while n < 18:
    grid[2,n] = abttns[n]
    n += 1
m=0
while m < 18:
    grid[3,m] = abttns[18+m]
    m += 1
k=0
while k < 18:
    grid[4,k] = abttns[36+k]
    k += 1
grid[5,0] = abttns[54]
grid[5,1] = abttns[55]
# grid[5,2] = the Lanthanide series hole
i=3
while i < 18:
    grid[5,i] = abttns[68+i]
    i += 1
grid[6,0] = abttns[86]
grid[6,1] = abttns[87]
# grid[6,2] = the Actinide series hole
j=2
while j < 17:
    # Lanthanide series
    grid[7,j] = abttns[54+j]
    j += 1
#Actinide series
grid[8,2] = abttns[88]
grid[8,3] = abttns[89]

# The periodic table button functions
output = widgets.Output()
#display(output)
@output.capture()

def on_a1_clicked(a1): selectAtom.value = 1 
def on_a2_clicked(a2): selectAtom.value = 2 
def on_a3_clicked(a3): selectAtom.value = 3 
def on_a4_clicked(a4): selectAtom.value = 4 
def on_a5_clicked(a5): selectAtom.value = 5 
def on_a6_clicked(a6): selectAtom.value = 6 
def on_a7_clicked(a7): selectAtom.value = 7 
def on_a8_clicked(a8): selectAtom.value = 8 
def on_a9_clicked(a9): selectAtom.value = 9 
def on_a10_clicked(a10): selectAtom.value = 10 
def on_a11_clicked(a11): selectAtom.value = 11 
def on_a12_clicked(a12): selectAtom.value = 12
def on_a13_clicked(a13): selectAtom.value = 13
def on_a14_clicked(a14): selectAtom.value = 14 
def on_a15_clicked(a15): selectAtom.value = 15 
def on_a16_clicked(a16): selectAtom.value = 16 
def on_a17_clicked(a17): selectAtom.value = 17 
def on_a18_clicked(a18): selectAtom.value = 18 
def on_a19_clicked(a19): selectAtom.value = 19 
def on_a20_clicked(a20): selectAtom.value = 20 
def on_a21_clicked(a21): selectAtom.value = 21 
def on_a22_clicked(a22): selectAtom.value = 22 
def on_a23_clicked(a23): selectAtom.value = 23 
def on_a24_clicked(a24): selectAtom.value = 24
def on_a25_clicked(a25): selectAtom.value = 25 
def on_a26_clicked(a26): selectAtom.value = 26
def on_a27_clicked(a27): selectAtom.value = 27
def on_a28_clicked(a28): selectAtom.value = 28
def on_a29_clicked(a29): selectAtom.value = 29 
def on_a30_clicked(a30): selectAtom.value = 30
def on_a31_clicked(a31): selectAtom.value = 31 
def on_a32_clicked(a32): selectAtom.value = 32
def on_a33_clicked(a33): selectAtom.value = 33
def on_a34_clicked(a34): selectAtom.value = 34
def on_a35_clicked(a35): selectAtom.value = 35
def on_a36_clicked(a36): selectAtom.value = 36
def on_a37_clicked(a37): selectAtom.value = 37
def on_a38_clicked(a38): selectAtom.value = 38
def on_a39_clicked(a39): selectAtom.value = 39
def on_a40_clicked(a40): selectAtom.value = 40 
def on_a41_clicked(a41): selectAtom.value = 41 
def on_a42_clicked(a42): selectAtom.value = 42
def on_a43_clicked(a43): selectAtom.value = 43
def on_a44_clicked(a44): selectAtom.value = 44 
def on_a45_clicked(a45): selectAtom.value = 45 
def on_a46_clicked(a46): selectAtom.value = 46 
def on_a47_clicked(a47): selectAtom.value = 47 
def on_a48_clicked(a48): selectAtom.value = 48 
def on_a49_clicked(a49): selectAtom.value = 49
def on_a50_clicked(a50): selectAtom.value = 50
def on_a51_clicked(a51): selectAtom.value = 51 
def on_a52_clicked(a52): selectAtom.value = 52 
def on_a53_clicked(a53): selectAtom.value = 53 
def on_a54_clicked(a54): selectAtom.value = 54 
def on_a55_clicked(a55): selectAtom.value = 55 
def on_a56_clicked(a56): selectAtom.value = 56 
def on_a57_clicked(a57): selectAtom.value = 57 
def on_a58_clicked(a58): selectAtom.value = 58 
def on_a59_clicked(a59): selectAtom.value = 59 
def on_a60_clicked(a59): selectAtom.value = 60 
def on_a61_clicked(a61): selectAtom.value = 61 
def on_a62_clicked(a62): selectAtom.value = 62 
def on_a63_clicked(a63): selectAtom.value = 63 
def on_a64_clicked(a64): selectAtom.value = 64 
def on_a65_clicked(a65): selectAtom.value = 65 
def on_a66_clicked(a66): selectAtom.value = 66 
def on_a67_clicked(a67): selectAtom.value = 67 
def on_a68_clicked(a68): selectAtom.value = 68 
def on_a69_clicked(a69): selectAtom.value = 69 
def on_a70_clicked(a70): selectAtom.value = 70 
def on_a71_clicked(a71): selectAtom.value = 71 
def on_a72_clicked(a72): selectAtom.value = 72 
def on_a73_clicked(a73): selectAtom.value = 73 
def on_a74_clicked(a74): selectAtom.value = 74 
def on_a75_clicked(a75): selectAtom.value = 75 
def on_a76_clicked(a76): selectAtom.value = 76 
def on_a77_clicked(a77): selectAtom.value = 77 
def on_a78_clicked(a78): selectAtom.value = 78 
def on_a79_clicked(a79): selectAtom.value = 79 
def on_a80_clicked(a80): selectAtom.value = 80 
def on_a81_clicked(a81): selectAtom.value = 81 
def on_a82_clicked(a82): selectAtom.value = 82 
def on_a83_clicked(a83): selectAtom.value = 83 
def on_a84_clicked(a84): selectAtom.value = 84 
def on_a85_clicked(a85): selectAtom.value = 85 
def on_a86_clicked(a86): selectAtom.value = 86 
def on_a87_clicked(a87): selectAtom.value = 87 
def on_a88_clicked(a88): selectAtom.value = 88 
def on_a89_clicked(a89): selectAtom.value = 89 
def on_a90_clicked(a90): selectAtom.value = 90     
    
a1.on_click(on_a1_clicked)
a2.on_click(on_a2_clicked)
a3.on_click(on_a3_clicked)
a4.on_click(on_a4_clicked)
a5.on_click(on_a5_clicked)
a6.on_click(on_a6_clicked)
a7.on_click(on_a7_clicked)
a8.on_click(on_a8_clicked)
a9.on_click(on_a9_clicked)
a10.on_click(on_a10_clicked)
a11.on_click(on_a11_clicked)
a12.on_click(on_a12_clicked)
a13.on_click(on_a13_clicked)
a14.on_click(on_a14_clicked)
a15.on_click(on_a15_clicked)
a16.on_click(on_a16_clicked)
a17.on_click(on_a17_clicked)
a18.on_click(on_a18_clicked)
a19.on_click(on_a19_clicked)
a20.on_click(on_a20_clicked)
a21.on_click(on_a21_clicked)
a22.on_click(on_a22_clicked)
a23.on_click(on_a23_clicked)
a24.on_click(on_a24_clicked)
a25.on_click(on_a25_clicked)
a26.on_click(on_a26_clicked)
a27.on_click(on_a27_clicked)
a28.on_click(on_a28_clicked)
a29.on_click(on_a29_clicked)
a30.on_click(on_a30_clicked)
a31.on_click(on_a31_clicked)
a32.on_click(on_a32_clicked)
a33.on_click(on_a33_clicked)
a34.on_click(on_a34_clicked)
a35.on_click(on_a35_clicked)
a36.on_click(on_a36_clicked)
a37.on_click(on_a37_clicked)
a38.on_click(on_a38_clicked)
a39.on_click(on_a39_clicked)
a40.on_click(on_a40_clicked)
a41.on_click(on_a41_clicked)
a42.on_click(on_a42_clicked)
a43.on_click(on_a43_clicked)
a44.on_click(on_a44_clicked)
a45.on_click(on_a45_clicked)
a46.on_click(on_a46_clicked)
a47.on_click(on_a47_clicked)
a48.on_click(on_a48_clicked)
a49.on_click(on_a49_clicked)
a50.on_click(on_a50_clicked)
a51.on_click(on_a51_clicked)
a52.on_click(on_a52_clicked)
a53.on_click(on_a53_clicked)
a54.on_click(on_a54_clicked)
a55.on_click(on_a55_clicked)
a56.on_click(on_a56_clicked)
a57.on_click(on_a57_clicked)
a58.on_click(on_a58_clicked)
a59.on_click(on_a59_clicked)
a60.on_click(on_a60_clicked)
a61.on_click(on_a61_clicked)
a62.on_click(on_a62_clicked)
a63.on_click(on_a63_clicked)
a64.on_click(on_a64_clicked)
a65.on_click(on_a65_clicked)
a66.on_click(on_a66_clicked)
a67.on_click(on_a67_clicked)
a68.on_click(on_a68_clicked)
a69.on_click(on_a69_clicked)
a70.on_click(on_a70_clicked)
a71.on_click(on_a71_clicked)
a72.on_click(on_a72_clicked)
a73.on_click(on_a73_clicked)
a74.on_click(on_a74_clicked)
a75.on_click(on_a75_clicked)
a76.on_click(on_a76_clicked)
a77.on_click(on_a77_clicked)
a78.on_click(on_a78_clicked)
a79.on_click(on_a79_clicked)
a80.on_click(on_a80_clicked)
a81.on_click(on_a81_clicked)
a82.on_click(on_a82_clicked)
a83.on_click(on_a83_clicked)
a84.on_click(on_a84_clicked)
a85.on_click(on_a85_clicked)
a86.on_click(on_a86_clicked)
a87.on_click(on_a87_clicked)
a88.on_click(on_a88_clicked)
a89.on_click(on_a89_clicked)
a90.on_click(on_a90_clicked)


In [25]:
# Defining the gui widgets
selectAtom = widgets.Dropdown(
    options= elements,
    value=2, 
    description='Atom:',
)
atomsLabel = widgets.Checkbox(
    value=False,
    description='Atomic label',
    disabled=False,
    indent=False
)
emissionType= widgets.Dropdown(
    options=['plane', 'cones'],
    value='plane', 
    description='Emission:',
)
coloredEmissions = widgets.Checkbox(
    value=False,
    description='Color emissions',
    disabled=False,
    indent=False,
)
# Changes the equatorial(or cone) emission radius
emRSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 8.0,
    step= 0.1,
    description='EmRadius',
)
# Changes the distance between adjacent orthogonal protons
ePSlider = widgets.FloatSlider(
    value = 5.0,
    min = 2.0,
    max = 7.0,
    step= 0.1,
    description='Equ1_Pole2',
)
# Changes the distance tween top and bottom alpha parallel protons
pPSlider = widgets.FloatSlider(
    value = 4.0,
    min = 3.0,
    max = 6.0,
    step= 0.1,
    description='A width',
)
# Changes the pole to pole distance between 2 adjacent alphas
p2p3Slider = widgets.FloatSlider(
    value = 3.0,
    min = 2.0,
    max = 5.0,
    step= 0.1,
    description='A1_A2 Sep',
)
# Changes the cone emission angle
coneAngleSlider = widgets.FloatSlider(
    value = 30,  # deg  
    min = 20,   
    max = 40,  
    step= 0.1,
    description='Cone angle',
)

# The guiwidget control panel 
guiwidgets = widgets.VBox([selectAtom,atomsLabel,emissionType, # moved
              coloredEmissions,emRSlider,ePSlider,
              pPSlider,p2p3Slider,coneAngleSlider])

# Observe any changes in widget values, button or gui
number = selectAtom.value
def handle_change(change):
    global number
    number = change.new
selectAtom.observe(handle_change, 'value')

emR = emRSlider.value
def handle_change(change):
    global emR
    emR = change.new
emRSlider.observe(handle_change, 'value');

ortho = ePSlider.value # proton1 equator to proton2 pole
def handle_change(change):
    global ortho
    ortho = change.new
ePSlider.observe(handle_change, 'value');

pPTP = pPSlider.value  # proton pole to proton pole 
def handle_change(change):
    global pPTP
    pPTP = change.new
pPSlider.observe(handle_change, 'value')

p2p3Dist = p2p3Slider.value # between 2 adjacent alphas
def handle_change(change):
    global p2p3Dist
    p2p3Dist = change.new
p2p3Slider.observe(handle_change, 'value')

cAngleDeg = coneAngleSlider.value # cone angle
def handle_change(change):
    global cAngleDeg
    cAngleDeg = change.new
coneAngleSlider.observe(handle_change, 'value')

The Periodic Table. The periodic table below is provided as a sensible way to select the desired atom, just click on it. The many atomic properties the table is known to provide may be taken into consideration while making a choice.

Most of the controls will be displayed in the cell following the periodic table. Note that within those controls, the atom can also be selected by a dropdown widget.

Other control options include: type of proton emissions displayed, color coding of the specific number of protons in each proton stack, proton separation distances, and whether to include an atomic label or not.

In [29]:
grid # Display the Periodic Table

GridspecLayout(children=(Button(description='H', layout=Layout(border='1px solid black', grid_area='widget001'…

In [30]:
guiwidgets # Display the main control widgets. 

In [31]:
# After making an element change, re-run this cell 
#and the rest.

An = elements[number-1]

atomicLabel = An[0] 

atomicLabel

' 24 chromium Cr '


Charge Field physics and the Unified Field theory, (and much more) have been well developed and well described by Miles Mathis. See,

***THE GREATEST STANDING ERRORS IN PHYSICS AND MATHEMATICS***
<http://milesmathis.com/index.html>
[Miles Mathis science site Homepage](http://milesmathis.com/index.html).

**SECTION 9: THE NUCLEUS** contains descriptions and diagrams 
of charge channeling and charge recycling by the 
elements. A paper most relevant to Atom Builder is 
>**How to Build the Elements**. 
>Explaining the periodic table, with nuclear diagrams".  
<http://milesmathis.com/nuclear.pdf>
[How to Build the Elements](http://milesmathis.com/nuclear.pdf)
For Charge Field discussion visit the forum at "Miles Mathis' Charge Field - Portal" https://milesmathis.forumotion.com.

This particular project is described in the Miles Periodic Table with Standard Periodic Table reference thread, https://milesmathis.forumotion.com/t634p75-miles-periodic-table-with-standard-periodic-table-reference#6702.

This project was Cr6's idea. He's also responsible for following Miles Mathis' atomic model and creating the Slotlayout diagram this project greatly relies.

There's at least one 10 year old charge field "Atom Viewer" out there, Nevyn's object oriented, way sophisticated javaScript code. I'll try to figure it out.

Please pardon my understandings, personal interpretations and mistakes. Feel free to make this project better.